<a href="https://colab.research.google.com/github/dakilaledesma/arcface-classifier/blob/main/notebooks/ArcFace_TF2_LayerImp_Augmentations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 123 ms, sys: 26 ms, total: 149 ms
Wall time: 20.3 s


In [2]:
! pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.3 MB/s 


In [3]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Dense, Concatenate, Flatten, GlobalAveragePooling2D, Input
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

import numpy as np

import tensorflow_addons as tfa
import tensorflow as tf

from tqdm.notebook import tqdm
from glob import glob
import os

In [4]:
# From https://github.com/ozora-ogino/asoftmax-tf/blob/main/asoftmax.py
class ASoftmax(tf.keras.layers.Layer):
    def __init__(
        self,
        n_classes=10,
        scale=30.0,
        margin=0.50,
        regularizer=None,
        **kwargs,
    ):
        """[ASoftmax]
        Args:
            n_classes (int, optional): Number of class. Defaults to 10.
            scale (float, optional): Float variable for scaling. Defaults to 30.0.
            margin (float, optional): Float variable of margin. Defaults to 0.50.
            regularizer (function, optional): keras.regularizers. Defaults to None.
        """

        super(ASoftmax, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.scale = scale
        self.margin = margin
        self.regularizer = regularizers.get(regularizer)

    def build(self, input_shape):
        super(ASoftmax, self).build(input_shape[0])
        print(input_shape)
        self.W = self.add_weight(
            name="W",
            shape=(input_shape[0][-1], self.n_classes),
            initializer="glorot_uniform",
            trainable=True,
            regularizer=self.regularizer,
        )

    def _train_op(self, inputs):
        x, y = inputs

        # Normalization
        x = tf.nn.l2_normalize(x, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)

        # Dot product
        logits = x @ W

        # Add margin and clip logits to prevent zero division when backward
        theta = tf.acos(K.clip(logits, -1.0 + K.epsilon(), 1.0 - K.epsilon()))
        target_logits = tf.cos(theta + self.margin)
        logits = logits * (1 - y) + target_logits * y

        # Rescale the feature
        logits *= self.scale
        out = tf.nn.softmax(logits)
        return out

    def _predict_op(self, inputs):
        # Normalization
        x = tf.nn.l2_normalize(inputs, axis=1)
        W = tf.nn.l2_normalize(self.W, axis=0)
        logits = x @ W
        out = tf.nn.softmax(logits)
        return out

    def call(self, inputs, training=False):
        if training:
            out = self._train_op(inputs)
        else:
            out = self._predict_op(inputs)
        return out

In [5]:
num_classes = 300

In [6]:
train_datagen = image.ImageDataGenerator(
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest',
      rescale=1./255)

In [7]:
# https://stackoverflow.com/questions/59921314/multi-input-model-with-flow-from-directory
def af_generator(datagen, directory, target_size, batch_size):
  gen = train_datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical')
  
  while True:
    x = gen.next()
    yield [x[0], x[1]], x[1]

In [8]:
train_generator = af_generator(
    train_datagen,
    'orchidaceae_train/',
    (224, 224),
    1)

In [9]:
val_generator = train_datagen.flow_from_directory(
    'orchidaceae_train/',
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

Found 9419 images belonging to 300 classes.


In [10]:
x = []
y = []
for i in tqdm(range(47095), total=47095):
  xv, yv = next(val_generator)
  x.append(xv[0])
  y.append(yv[0])

  0%|          | 0/47095 [00:00<?, ?it/s]

In [11]:
class AFModel(Model):
  def __init__(self, num_classes=300, weight_decay=1e-4):
        super(AFModel, self).__init__()
        self.label_input = Input(shape=(num_classes,))
        self.backbone = ResNet50(
            input_shape=(224, 224, 3), 
            weights='imagenet', 
            include_top=False)
        self.layer_1 = GlobalAveragePooling2D()
        self.layer_2 = Dense(512, activation='relu')

        self.out = ASoftmax(
            n_classes=num_classes,
            regularizer=regularizers.l2(weight_decay),
        )

  def call(self, x, training=False):
      if training:
          x, y = x[0], x[1]
      x = self.backbone(x)
      x = self.layer_1(x)
      x = self.layer_2(x)

      if training:
          # When training, you need to pass label to ASoftmax
          out = self.out([x, y])
      else:
          out = self.out(x)
      return out

model = AFModel()

opt = tfa.optimizers.AdaBelief(learning_rate=1e-3)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(
      x=[x, y],
      y=y,
      epochs=100,
      verbose=2)

94781440/94765736 [==============================] - 1s 0us/step


InternalError: ignored

In [ ]:
%%time
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_test.zip -d /content/

In [ ]:
test_x = []
test_y = []
orc_test = sorted(glob("orchidaceae_test/**/*.*", recursive=True))
for fn in tqdm(orc_test, total=len(list(orc_test))):
  bn =  os.path.basename(fn)
  cat = cat_to_int[fn.split("/")[-2]]

  img = image.load_img(fn, target_size=(224, 224))
  x = image.img_to_array(img)
  x = preprocess_input(x)
  test_x.append(x)

  test_y.append(cat)

In [ ]:
test_x = np.array(test_x)

In [ ]:
preds = []
for idx, im in tqdm(enumerate(test_x), total=len(test_x)):
  im = np.expand_dims(im, 0)
  pred = np.argmax(model(im))
  preds.append(pred)

In [ ]:
# Accuracy on training data
print(accuracy_score(test_y, preds))

In [ ]:
# Accuracy on test data
print(accuracy_score(test_y, preds))

In [ ]:
model.save_weights("arcface_orcl_1_max")